In [174]:
class Tile:
    def __init__(self, i, data):
        self.id = int(i.split(" ")[1][:-1])
        self.data = data
        self.size = len(data)
        edge1 = data[0]
        edge2 = data[self.size - 1]
        edge3 = "".join(list(map(lambda x: x[0], data)))
        edge4 = "".join(list(map(lambda x: x[self.size -1], data)))
        self.edges = set([
            edge1, edge2, edge3, edge4,
            edge1[::-1], edge2[::-1], edge3[::-1], edge4[::-1],
        ])
        
    def flip(self):
        self.data = [x[::-1] for x in self.data]
    
    def rotate(self):
        new_data = []
        for i in range(self.size - 1 , -1, -1):
            new_data.append("".join(list(map(lambda x: x[i], self.data))))
        self.data = new_data
        
    def top(self):
        return self.data[0]
    
    def bottom(self):
        return self.data[self.size - 1]
    
    def left(self):
        return "".join(list(map(lambda x: x[0], self.data)))
    
    def right(self):
        return "".join(list(map(lambda x: x[self.size - 1], self.data)))

In [200]:
input_data = open('inputs/20.txt', 'r').read().split('\n\n')
input_data = [x.split('\n') for x in input_data]
tiles = {int(x[0].split(" ")[1][:-1]): Tile(x[0], x[1:]) for x in input_data}

In [211]:
neighbors = {tiles[tile].id: [] for tile in tiles}
for i in tiles:
    for j in tiles:
        if i != j:
            intersection = tiles[i].edges & tiles[j].edges
            if intersection != set():
                neighbors[tiles[i].id].append(tiles[j].id)

In [212]:
neighbors

{2411: [1453, 1489, 1613, 2017],
 1997: [3877, 3719, 1409, 3767],
 1427: [3163, 1801, 2647, 2789],
 2161: [2389, 1231, 3803],
 1321: [1823, 2473, 1613],
 1181: [2113, 1187, 2267, 2851],
 2749: [3257, 2711, 2131, 2347],
 3911: [1117, 3389, 2819, 3259],
 3257: [2749, 2087, 3407, 1361],
 2237: [3209, 1409, 3767],
 2389: [2161, 3347, 2963, 1993],
 3209: [2237, 1193],
 2579: [2113, 2633, 3023],
 2087: [3257, 2711, 3673, 2417],
 3517: [3719, 1979, 2917, 2351],
 3347: [2389, 1231, 2383, 3793],
 2711: [2749, 2087, 3067, 3307],
 3877: [1997, 2549, 1153, 3673],
 1721: [1033, 1787, 1259, 1031],
 3457: [3511, 2917, 2281, 3449],
 1231: [2161, 3347, 2473],
 1901: [3499, 2591, 2539, 2239],
 3329: [2659, 3547, 2137],
 1291: [2659, 3547, 3169, 2521],
 1481: [2297, 2083, 1237, 1777],
 2063: [1663, 3931, 2441],
 2297: [1481, 2969, 1259, 2539],
 2731: [1123, 2729, 1987, 2381],
 1453: [2411, 3191, 2897, 2707],
 1117: [3911, 3571, 1171, 1693],
 1823: [1321, 1279, 2897],
 3163: [1427, 3323, 3571, 1171],
 208

In [162]:
count = 0
value = 1
corners = []
for key in neighbors.keys():
    if len(neighbors[key]) == 2:
        count += 1
        value *= key
        corners.append(key)
count, value

(4, 63187742854073)

In [163]:
corners

[3209, 3803, 1399, 3701]

In [164]:
len(tiles)

144

In [266]:
complete_tiles = []
def orient_neighbors(tile_id):
    complete_tiles.append(tile_id)
    tile = tiles[tile_id]
    for x in neighbors[tile_id]:
        if x not in complete_tiles:
            x_tile = tiles[x]
            line_up(tile, x_tile)
            orient_neighbors(x_tile.id)
        else:
            pass

In [267]:
def line_up(correct_tile, new_tile):
    count = 0
    while not is_lined_up(correct_tile, new_tile):
        print(count)
        if count > 9:
            raise Error("Too many checks!")
        elif count == 4:
            new_tile.flip()
        else:
            new_tile.rotate()
        count += 1

In [268]:
def is_lined_up(tile_a, tile_b):
    return (tile_a.top() == tile_b.bottom()) or (tile_a.left() == tile_b.right()) or (tile_a.bottom() == tile_b.top()) or (tile_a.right() == tile_b.left())

In [269]:
orient_neighbors(corner.id)

In [264]:
def build_puzzle(corner_id):
    current_tile_id = corner_id
    puzzle = []
    for i in range(12):
        row = []
        for j in range(12):
            row.append(current_tile_id)
            for n in neighbors[current_tile_id]:
                if tiles[current_tile_id].right() == tiles[n].left():
                    current_tile_id = n
        puzzle.append(row)
        left_piece_id = row[0]
        for n in neighbors[left_piece_id] :
            if tiles[left_piece_id].bottom() == tiles[n].top():
                current_tile_id = n
    return puzzle

In [276]:
puzzle = build_puzzle(3209)

In [271]:
t = tiles[3209]
t.data

['..##.#####',
 '.........#',
 '##..#.##..',
 '...#.#...#',
 '....#.#..#',
 '#...#.....',
 '#.#.#.....',
 '#...#....#',
 '#.#......#',
 '...#..####']

In [281]:
def remove_border(tile):
    return [ row[1:-1] for row in tile.data[1:-1]]

In [310]:
def concat_data(puzzle):
    data = []
    for i in range(len(puzzle)):
        row_data = []
        for j in range(len(puzzle[0])):
            row_data.append(remove_border(tiles[puzzle[i][j]]))
        concat_data = [''.join(x) for x in list(zip(*row_data))]
        data = data + concat_data
    return data

In [311]:
d = concat_data(puzzle)

In [317]:
Picture = Tile("Tile 0:", d)

In [319]:
Picture.data

['........#.#.....#..#.#.#...#.#..#...#..#......#..###.....#.#....#.........##..#...#.......#..#..',
 '#..#.##......#...####..#...#.........#.#.#....#.#.....#..#.........#.##.#..#......#...#..#.#.###',
 '..#.#...#......#....#.....#......#.#..#.#...###.....##...#........#...#........##..#.....#..#...',
 '...#.#......######...#.....#...#...#...............#........#.#...##.#...#....#.#...#.#...#.....',
 '...#....#.#....###..............#...#.......................#..#........#.##.......#......#...#.',
 '.#.#......#..#.......##.###.#....#.............##..####..##.####...#...#..#....##...#.#.#.......',
 '...#......#.....#...##.#.....#....##...#.......##...###..###.##.####.#...##.##.#..##.#..#......#',
 '.#......#....##...###..#.......#.#...##......##.......#..........#......#.#..##..#.##.#.####...#',
 '....#....#...#..........#..........##...#..#.#..#.#...#...##.........#......#.#......#....##....',
 '...#........#...#.......##..#..#.........#........#...##..#.#.#.##..#.##....#.....#.##.##

In [320]:
Picture.flip()

In [322]:
Picture.data

['..#..#.......#...#..##.........#....#.#.....###..#......#..#...#..#.#...#.#.#..#.....#.#........',
 '###.#.#..#...#......#..#.##.#.........#..#.....#.#....#.#.#.........#...#..####...#......##.#..#',
 '...#..#.....#..##........#...#........#...##.....###...#.#..#.#......#.....#....#......#...#.#..',
 '.....#...#.#...#.#....#...#.##...#.#........#...............#...#...#.....#...######......#.#...',
 '.#...#......#.......##.#........#..#.......................#...#..............###....#.#....#...',
 '.......#.#.#...##....#..#...#...####.##..####..##.............#....#.###.##.......#..#......#.#.',
 '#......#..#.##..#.##.##...#.####.##.###..###...##.......#...##....#.....#.##...#.....#......#...',
 '#...####.#.##.#..##..#.#......#..........#.......##......##...#.#.......#..###...##....#......#.',
 '....##....#......#.#......#.........##...#...#.#..#.#..#...##..........#..........#...#....#....',
 '#.#....##.##.#.....#....##.#..##.#.#.#..##...#........#.........#..#..##.......#...#.....

In [330]:
def match_monster(region):
    first = region[0][18] == '#'
    second = (region[1][0] == '#') and (region[1][5] == '#') and (region[1][6] == '#') and (region[1][11] == '#') and (region[1][12] == '#')  and (region[1][17] == '#')  and (region[1][18] == '#')  and (region[1][19] == '#')
    third = (region[2][1] == '#') and (region[2][4] == '#') and (region[2][7] == '#') and (region[2][10] == '#') and (region[2][13] == '#')  and (region[2][16] == '#')
    return first and second and third

In [370]:
Picture.flip()

In [371]:
matches = []
for i in range(Picture.size - 3):
    for j in range(Picture.size - 20):
        r = [row[j:j+20] for row in Picture.data[i: i+3]]
        is_monster = match_monster(r)
        matches.append(is_monster)
print(sum(matches))

18


In [372]:
total_count = 0
for x in Picture.data:
    total_count += x.count("#")

In [373]:
total_count

2422

In [374]:
2422-15*18

2152